# BIC - CW

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [806 'France' 'Male' ... 1 1 142838.64]
 [757 'Germany' 'Male' ... 1 1 127059.04]
 [570 'France' 'Female' ... 0 1 116503.92]]


In [4]:
print(y)

[1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0
 0 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 1 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 

### Encoding categorical data

Label Encoding the "Gender" column

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [6]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [806 'France' 1 ... 1 1 142838.64]
 [757 'Germany' 1 ... 1 1 127059.04]
 [570 'France' 0 ... 0 1 116503.92]]


One Hot Encoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 1 1 142838.64]
 [0.0 1.0 0.0 ... 1 1 127059.04]
 [1.0 0.0 0.0 ... 0 1 116503.92]]


### Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN and adding custom Gaussian, Cosine and Null Activation Functions

In [11]:
ann = tf.keras.models.Sequential()
K = tf.keras.backend
utils = tf.keras.utils
def gaussian(x):
    return K.exp(-K.pow(x,2)/2)
def cosine(x) :
    return K.cos(x)
def nullact(x) :
    return K.constant(0)*x

utils.get_custom_objects().update({'gaussian': tf.keras.layers.Activation(gaussian)})
utils.get_custom_objects().update({'cosine': tf.keras.layers.Activation(cosine)})
utils.get_custom_objects().update({'nullact': tf.keras.layers.Activation(nullact)})

### Number of Neurons and Layers Configuration

In [12]:
neurons = int(input("Enter number of neurons: "))

Enter number of neurons: 6


In [13]:
layers = []
n = int(input("Enter number of layers (1-5): "))
if n < 6 and n > 0:
    for i in range (0, n):
        print("Hyperbolic tangent layer must run every time.")
        print("Available activation functions: linear, sigmoid, gaussian, cosine, null")
        layer = input("Enter name of desired activation function: ")
    
        layers.append(layer)    
else:
    print("Wrong number of layers entered")

Enter number of layers (1-5): 1
Hyperbolic tangent layer must run every time.
Available activation functions: linear, sigmoid, gaussian, cosine, null
Enter name of desired activation function: sigmoid


### Adding the input layer and the first hidden layer with linear AF

In [14]:
if "linear" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

### Adding the second hidden layer with Sigmoid AF

In [15]:
if "sigmoid" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='sigmoid'))

### Adding the third hidden layer with Gaussian AF

In [16]:
if "gaussian" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='gaussian'))

### Adding the fourth hidden layer with Cosine AF

In [17]:
if "cosine" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='cosine'))

### Adding the fifth hidden layer with Null AF

In [18]:
if "null" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='nullact'))

### Adding the output layer with Hyperbolic Tangent AF

In [19]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Part 3 - Training the ANN

### Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [21]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
13/13 [==============================] - 0s 693us/step - loss: 2.5573 - accuracy: 0.7950
Epoch 2/100
13/13 [==============================] - 0s 760us/step - loss: 2.4961 - accuracy: 0.7975
Epoch 3/100
13/13 [==============================] - 0s 664us/step - loss: 2.4564 - accuracy: 0.7975
Epoch 4/100
13/13 [==============================] - 0s 563us/step - loss: 2.3925 - accuracy: 0.7975
Epoch 5/100
13/13 [==============================] - 0s 529us/step - loss: 2.3311 - accuracy: 0.8000
Epoch 6/100
13/13 [==============================] - 0s 606us/step - loss: 2.2924 - accuracy: 0.8000
Epoch 7/100
13/13 [==============================] - 0s 634us/step - loss: 2.2279 - accuracy: 0.8025
Epoch 8/100
13/13 [==============================] - 0s 558us/step - loss: 2.2227 - accuracy: 0.8025
Epoch 9/100
13/13 [==============================] - 0s 586us/step - loss: 2.2179 - accuracy: 0.8025
Epoch 10/100
13/13 [==============================] - 0s 663us/step - loss: 2.1914 - accura

13/13 [==============================] - 0s 517us/step - loss: 1.4271 - accuracy: 0.8025
Epoch 82/100
13/13 [==============================] - 0s 494us/step - loss: 1.4270 - accuracy: 0.8025
Epoch 83/100
13/13 [==============================] - 0s 539us/step - loss: 1.4269 - accuracy: 0.8025
Epoch 84/100
13/13 [==============================] - 0s 484us/step - loss: 1.4268 - accuracy: 0.8025
Epoch 85/100
13/13 [==============================] - 0s 467us/step - loss: 1.4267 - accuracy: 0.8025
Epoch 86/100
13/13 [==============================] - 0s 500us/step - loss: 1.4266 - accuracy: 0.8025
Epoch 87/100
13/13 [==============================] - 0s 518us/step - loss: 1.4265 - accuracy: 0.8025
Epoch 88/100
13/13 [==============================] - 0s 480us/step - loss: 1.4264 - accuracy: 0.8025
Epoch 89/100
13/13 [==============================] - 0s 464us/step - loss: 1.4263 - accuracy: 0.8025
Epoch 90/100
13/13 [==============================] - 0s 498us/step - loss: 1.4262 - accuracy: 

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Solution**

In [22]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


### Predicting the Test set results

In [23]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [1 1]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [0 0]
 [1 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 0]
 [0 0]
 [0 1]]


### Making the Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[73  7]
 [17  3]]


0.76